In [ ]:
import sys
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [ ]:
DATALOADER_DIR = '/content/drive/MyDrive/AI_Master/Data Mining/parquets'
BOOK_VAL = { 0: 'search', 1: 'click', 2: 'book'}


In [ ]:
def load_train_test_dfs():
    train_df = pd.read_parquet(f'{DATALOADER_DIR}/train_df.parquet')
    test_df = pd.read_parquet(f'{DATALOADER_DIR}/test_df.parquet')
    return train_df, test_df

In [ ]:
train_df, test_df = load_train_test_dfs()

### Statistics

In [ ]:
# Records
print(f"Total Records: {train_df.shape[0]}")
print(f"Total Searches: {train_df['srch_id'].nunique()}")
print(f"Total Clicks: {round(train_df['click_bool'].sum()/train_df.shape[0]*100, 2)}%")
print(f"Total Bookings: {round(train_df['booking_bool'].sum()/train_df.shape[0]*100, 2)}%")
print(f"Books per Click: {round(train_df['booking_bool'].sum()/train_df['click_bool'].sum()*100, 2)}%")

In [ ]:
# Searches
print(f"Average Hotel Results per Search: {round(np.mean(train_df.groupby('srch_id')['prop_id'].count()), 2)}")
print(f"Std Hotel Results per Search: {round(np.std(train_df.groupby('srch_id')['prop_id'].count()), 2)}")
print(f"Median Hotel Results per Search: {round(np.median(train_df.groupby('srch_id')['prop_id'].count()), 2)}")

# print(f"Average Destinations per Search: {round(np.mean(train_df.groupby('srch_id')['srch_destination_id'].count()), 2)}")
# print(f"Std Destinations per Search: {round(np.std(train_df.groupby('srch_id')['srch_destination_id'].count()), 2)}")
# print(f"Median Destinations per Search: {round(np.median(train_df.groupby('srch_id')['srch_destination_id'].count()), 2)}")

print(f"Average Clicks per Search: {round(np.mean(train_df.groupby('srch_id')['click_bool'].sum()), 2)}")
print(f"Average Books per Search: {round(np.mean(train_df.groupby('srch_id')['booking_bool'].sum()), 2)}")

In [ ]:
# Categories
print(f"Properties: {train_df['prop_id'].nunique()}")
print(f"Countries: {train_df['prop_country_id'].nunique()}")
print(f"Destinations: {train_df['srch_destination_id'].nunique()}")
print(f"Visitor Countries: {train_df['visitor_location_country_id'].nunique()}")
print(f"Brand Hotels: {train_df['prop_brand_bool'].sum()}")



### Missing Values

In [ ]:
def create_missingvalues(df: pd.DataFrame) -> None:
    count = df.shape[0]
    missing_df = df.isnull().sum()
    missing_df = missing_df/count
    missing_df = missing_df[missing_df!= 0].sort_values()

    fig = px.bar(y=missing_df.values, x=missing_df.index, labels={'y':'Missing values (%)'})
    fig.update_traces(textfont_size=12, textangle=-45, textposition="outside", cliponaxis=True, marker_color='DarkSlateGrey')
    fig.show(scale=2)


create_missingvalues(train_df)

### Boxplots

In [ ]:
create_boxplot_features = ['price_usd', 'orig_destination_distance',
    'comp1_rate_percent_diff', 'comp2_rate_percent_diff', 
    'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff',
    'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff']

extra_features=['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_review_score','prop_location_score1',
    'prop_location_score2', 'prop_log_historical_price', 'srch_query_affinity_score',]
    
fig = go.Figure()
fig.update_xaxes(type="log")
for feature in create_boxplot_features:
    print(feature)
    fig.add_trace(go.Box(x=train_df[feature].dropna(), name=feature))

fig.show()
    
# fig.write_image('/content/drive/MyDrive/AI_Master/Data Mining/boxplots.png')

In [ ]:
fig.show()

### Temporal Data (log of values)

In [ ]:
def extract_booking_date(df: pd.DataFrame) -> pd.DataFrame:
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d')
    df['date'] = df['date_time'].apply(
        lambda date: str(date.year)+"-"+str(date.month))
    df['booking'] = df['booking_bool'] + df['click_bool']
    return df[['date', 'booking']]

df_date = extract_booking_date(train_df)

In [ ]:

data = df_date.groupby(['date','booking'])['booking'].count().reset_index(name='counts')
data['booking'] = data['booking'].apply(lambda x: BOOK_VAL[x])
fig1 = px.line(data[data['booking'] != 'search'], x="date", y="counts", color='booking', labels={'counts': 'Number of bookings/clicks', 'date': 'Date'})
fig1.update_layout(
    width=700, height=500)
fig1.show()

fig2 = px.line(data[data['booking'] == 'search'], x="date", y="counts", color='booking', color_discrete_map={BOOK_VAL[0]:'green'}, labels={'counts': 'Number of bookings/clicks', 'date': 'Date'})
fig2.update_layout(
    width=700, height=500)
fig2.show()

### Potition (2 plots -> random - not random)

In [ ]:
def extract_position(df: pd.DataFrame) -> pd.DataFrame:
  df['booking'] = df['booking_bool'] + df['click_bool']
  data = df.groupby(['position','booking', 'random_bool'])['booking'].count().reset_index(name='counts')
  data[data['booking'] == 1]['counts'] = data[data['booking'] == 1]['counts'] + data[data['booking'] == 2]['counts'] 
  data['booking'] = data['booking'].apply(lambda x: BOOK_VAL[x])

  return data[['position', 'booking', 'counts', 'random_bool']]

data = extract_position(train_df)

In [ ]:
data
# 2	1	click	2894	0
#3	1	click	7380	1
# 4	1	book	27014	0
# 5	1	book	1101	1

In [ ]:
import numpy as np

df = data[data['booking'] != BOOK_VAL[0]]
new_data = df.groupby(['position','random_bool'])['counts'].sum().reset_index()
new_data['booking'] = BOOK_VAL[1]
merged = pd.merge(df, new_data, how='left', on=['position','random_bool', 'booking'])
merged['counts'] = np.max(merged[['counts_x', 'counts_y']], axis=1)
merged


In [ ]:
# data = data[data['booking'] != BOOK_VAL[0]]
# merged['counts'] -= merged['counts'].min()  # equivalent to df = df - df.min()
# merged['counts'] /= merged['counts'].max()

fig = px.bar(merged[merged['random_bool'] > 0], x="position", y="counts", barmode='group', color='booking') #labels={'counts':'Bookings', 'travel_time':'Travel Month'})
fig.update_layout(
    width=700, height=500)
fig.show(scale=2)

fig2 = px.bar(merged[merged['random_bool'] < 1], x="position", y="counts", barmode='group', color='booking') #labels={'counts':'Bookings', 'travel_time':'Travel Month'})
fig2.update_layout(
    width=700, height=500)
fig2.show(scale=2)

###  First plot - prop_location_score2 (booked + clicked)

In [ ]:
import plotly.figure_factory as ff
import numpy as np

feature_name = 'prop_location_score2'

x1 = train_df[feature_name][(train_df["booking_bool"]==0) & (train_df[feature_name].notnull())]
x2 = train_df[feature_name][(train_df["booking_bool"]==1) & (train_df[feature_name].notnull())]
# x3 = train_df[feature_name][(train_df["click_bool"]==0) & (train_df[feature_name].notnull())]
# x4 = train_df[feature_name][(train_df["click_bool"]==1) & (train_df[feature_name].notnull())]


hist_data = [x1, x2] #x4]

group_labels = ['Not Booked', 'Booked'] #'Clicked']
colors = ['#333F44', '#37AA9C'] # '#835AF1' ]

# Create distplot with curve_type set to 'normal'
fig2 = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors, show_rug=False)
fig2.update_xaxes(showticklabels=True)
fig2.update_layout(
    xaxis_title="Desirability of hotel (2)",
    yaxis_title="Density",
    width=700, height=500)
fig2.show(scale=2)

### Second plot - price vs density (booked + clicked)

In [ ]:
import plotly.figure_factory as ff
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

# scale = Normalizer()

feature_name = 'price_usd'

x1 = train_df[feature_name][(train_df["booking_bool"]==0) & (train_df[feature_name].notnull()) & (train_df[feature_name]<1000)]
x2 = train_df[feature_name][(train_df["booking_bool"]==1) & (train_df[feature_name].notnull()) & (train_df[feature_name]<1000)]
# x3 = train_df[feature_name][(train_df["click_bool"]==0) & (train_df[feature_name].notnull())]
# x4 = train_df[feature_name][(train_df["click_bool"]==1) & (train_df[feature_name].notnull())]

# x1 = (x1 - x1.min()) / (x1.max() - x1.min())
# x2 = (x2 - x2.min()) / (x2.max() - x2.min())
# x1 = np.array(scale.fit_transform(x1)).flatten()
# x2 = np.array(scale.fit_transform(x2)).flatten()


hist_data = [x1, x2] #x4]

group_labels = ['Not Booked', 'Booked'] #'Clicked']
colors = ['#333F44', '#37AA9C'] # '#835AF1' ]

# Create distplot with curve_type set to 'normal'
fig_price = ff.create_distplot(hist_data, group_labels, show_hist=True, bin_size=0.1, colors=colors, show_rug=False)
fig_price.update_layout(
    xaxis_title="Hotel Price",
    yaxis_title="Density",
    width=700, height=500)
fig_price.show()

In [ ]:
np.array(x1).flatten()

In [ ]:
print(x1.shape[0], x2.shape[0])

### Expedia's price vs Competitors (Histogram plot with -1, 0, 1)

In [ ]:
def extract_competitors_price(df: pd.DataFrame, comp):
  # Create new col
  df['comp'] = df[comp].dropna()
  # df['comp'] = df['comp'].mask((df.filter(regex='comp._rate') < 0).any(axis=1), -1).mask(
  #   (df.filter(regex='comp._rate') > 0).any(axis=1), 1)
  df['booking'] = df['click_bool'] + df['booking_bool']
  data = df.groupby(['comp','booking'])['booking'].count().reset_index(name='counts')
  data['booking'] = data['booking'].apply(lambda x: BOOK_VAL[x])
  return data[['comp', 'booking', 'counts']]

def merge_booking(df: pd.DataFrame):
  df = df[data['booking'] != BOOK_VAL[0]]
  new_data = df.groupby(['comp'])['counts'].sum().reset_index()
  new_data['booking'] = BOOK_VAL[1]
  merged = pd.merge(df, new_data, how='left', on=['comp', 'booking'])
  merged['counts'] = np.max(merged[['counts_x', 'counts_y']], axis=1)
  return merged

df = train_df.copy()
merged_df = extract_competitors_price(df, f'comp1_rate')
for i in range(2, 9):
  data = extract_competitors_price(df, f'comp{i}_rate')
  merged_df=merged_df.append(data)

data2 = merge_booking(merged_df)
fig = px.bar(data2, x="comp", y="counts", color='booking', barmode='group', labels={'counts':'Number of Clicked or Booked', 'comp':'Expedia\'s price compared to competitors '})

fig.show(scale=2)

### Travelling Months (histigram)

In [ ]:
def extract_booking_travel_date(df: pd.DataFrame) -> pd.DataFrame:
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d')
    df['travel_time'] =  pd.to_datetime((df['date_time'] + pd.to_timedelta(df['srch_booking_window'], unit='D')))
    df['booking'] = df['booking_bool'] + df['click_bool']
    return df[['date_time','travel_time', 'booking']]

df_month = extract_booking_travel_date(train_df)

In [ ]:
import calendar

data = df_month.groupby(['travel_time','booking'])['booking'].count().reset_index(name='counts')
data['month'] = (data['travel_time'].dt.month).apply(lambda x: calendar.month_abbr[x])
data["travel_time"] = data["travel_time"].dt.strftime('2013-%m-%d')
fig = px.bar(data[data['booking'] > 1], x="travel_time", y="counts", color='month', labels={'counts':'Bookings', 'travel_time':'Travel Month'})
    # category_orders={'month': {
    # 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun' , 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'}})
fig.update_xaxes(dtick="M1", tickformat='%b', position=1)
fig.show(scale=2)

### Feature Importance

In [ ]:
train_df.columns

In [ ]:
# from sklearn.ensemble import ExtraTreesClassifier
# import matplotlib.pyplot as plt

# X = train_df.copy()  #independent columns
# y =  (train_df['booking_bool'] * 4 + train_df['click_bool']).round().astype('Int64')    #target column i.e price range
# model = ExtraTreesClassifier()
# model.fit(X,y)
# print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
# #plot graph of feature importances for better visualization
# feat_importances = pd.Series(model.feature_importances_, index=X.columns)
# feat_importances.nlargest(10).plot(kind='barh')
# plt.show()

In [13]:
np.array(x1).flatten()

array([-0.00933228, -0.00520756, -0.00464109, ..., -0.01031829,
       -0.00969304, -0.01206897])

In [ ]:
print(x1.shape[0], x2.shape[0])

4819957 138390


### Expedia's price vs Competitors (Histogram plot with -1, 0, 1)

In [ ]:
def extract_competitors_price(df: pd.DataFrame, comp):
  # Create new col
  df['comp'] = df[comp].dropna()
  # df['comp'] = df['comp'].mask((df.filter(regex='comp._rate') < 0).any(axis=1), -1).mask(
  #   (df.filter(regex='comp._rate') > 0).any(axis=1), 1)
  df['booking'] = df['click_bool'] + df['booking_bool']
  data = df.groupby(['comp','booking'])['booking'].count().reset_index(name='counts')
  data['booking'] = data['booking'].apply(lambda x: BOOK_VAL[x])
  return data[['comp', 'booking', 'counts']]

def merge_booking(df: pd.DataFrame):
  df = df[data['booking'] != BOOK_VAL[0]]
  new_data = df.groupby(['comp'])['counts'].sum().reset_index()
  new_data['booking'] = BOOK_VAL[1]
  merged = pd.merge(df, new_data, how='left', on=['comp', 'booking'])
  merged['counts'] = np.max(merged[['counts_x', 'counts_y']], axis=1)
  return merged

df = train_df.copy()
merged_df = extract_competitors_price(df, f'comp1_rate')
for i in range(2, 9):
  data = extract_competitors_price(df, f'comp{i}_rate')
  merged_df=merged_df.append(data)

data2 = merge_booking(merged_df)
fig = px.bar(data2, x="comp", y="counts", color='booking', barmode='group', labels={'counts':'Number of Clicked or Booked', 'comp':'Expedia\'s price compared to competitors '})

fig.show(scale=2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



### Travelling Months (histigram)

In [31]:
def extract_booking_travel_date(df: pd.DataFrame) -> pd.DataFrame:
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d')
    df['travel_time'] =  pd.to_datetime((df['date_time'] + pd.to_timedelta(df['srch_booking_window'], unit='D')))
    df['booking'] = df['booking_bool'] + df['click_bool']
    return df[['date_time','travel_time', 'booking']]

df_month = extract_booking_travel_date(train_df)

In [32]:
import calendar

data = df_month.groupby(['travel_time','booking'])['booking'].count().reset_index(name='counts')
data['month'] = (data['travel_time'].dt.month).apply(lambda x: calendar.month_abbr[x])
data["travel_time"] = data["travel_time"].dt.strftime('2013-%m-%d')
fig = px.bar(data[data['booking'] > 1], x="travel_time", y="counts", color='month', labels={'counts':'Bookings', 'travel_time':'Travel Month'})
    # category_orders={'month': {
    # 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun' , 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'}})
fig.update_xaxes(dtick="M1", tickformat='%b', position=1)
fig.show(scale=2)

### Feature Importance

In [6]:
train_df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [ ]:
# from sklearn.ensemble import ExtraTreesClassifier
# import matplotlib.pyplot as plt

# X = train_df.copy()  #independent columns
# y =  (train_df['booking_bool'] * 4 + train_df['click_bool']).round().astype('Int64')    #target column i.e price range
# model = ExtraTreesClassifier()
# model.fit(X,y)
# print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
# #plot graph of feature importances for better visualization
# feat_importances = pd.Series(model.feature_importances_, index=X.columns)
# feat_importances.nlargest(10).plot(kind='barh')
# plt.show()